# Criando a documentação automática usando Python

## Resumo do projeto e instruções:

---

Esse é um projeto de criação de documentação automática do Power BI utilizando o Python, a partir do arquivo .pbit de um projeto de Power BI.

É necessário, antes de executar o código:
- Transformar seu arquivo .pbix em .pbit, para fazer isso, basta abrir seu arquivo no Power BI Desktop, ir em Arquivo > Exportar > Modelo do Power BI;
- Ter o Python baixado na sua máquina e, opcionalmente, o VSCode ou outro editor de código (utilizei o Jupyter Notebook);
- Instalar e importar as bibliotecas necessárias (já estão no código);
- Baixar o arquivo modelo em word ou utilizar o seu próprio (configurando o código conforme seu próprio modelo).

*O resultado final será um arquivo Word com as informações de Páginas, Tabelas, Colunas, Medidas, Fontes e Relacionamentos de tabelas do projeto.*

Espero que goste do projeto e que ele te auxilie em sua jornada de inteligência de dados.

**Antes de executar o código, troque os caminhos das variáveis: caminho_BI, nome_BI, caminho_modelo_word e nome_modelo_word no arquivo config!**

## Primeira parte do código: 

Para a primeira parte do código, utilizo as bibliotecas json, pandas e zipfile para:
1. Transformar o arquivo .pbit em .zip
2. Extrair os arquivos "Layout" e "ModelSchema" em JSON
3. Retornar o arquivo .zip para .pbit
4. Extrair as informações de páginas e visuais do arquivo "Layout"
5. Extrair as informações de tabelas, medidas, fontes e relacionamentos do arquivo "ModelSchema"
6. Transformar todas as informações em dataframes utilizando a biblioteca pandas, para posterior exportação para o Word

In [ ]:
# Baixando as bibliotecas necessárias
!pip install -r requirements.txt

In [ ]:
# #Caso a linha anterior não funcione, descomente e use:
# !pip install pandas python-docx

In [ ]:
#Importando as bibliotecas
import json
import pandas as pd
import config
from os import path, rename
import zipfile
import datetime
from datetime import datetime
import requests
from io import StringIO
import time
import docx
from docx import Document

In [ ]:
# Traz a data atual para inserir no relatório
data_atual = datetime.now()

# Formata a data no formato desejado
data_formatada = data_atual.strftime("%d/%m/%Y")

print("Data da documentação:", data_formatada)

In [ ]:
# Substitua no arquivo .config os caminhos e nomes do relatório e modelo

caminho_BI = config.caminho_BI #Caminho do arquivo do seu relatório
nome_BI = config.nome_BI #Nome do arquivo do seu relatório

# Substitua pelo caminho e nome do arquivo do seu modelo do word (ou o baixado)
caminho_modelo_word = config.caminho_modelo_word #Caminho onde está seu modelo word (sem incluir o modelo)
nome_modelo_word = config.nome_modelo_word #Nome do seu arquivo modelo word

arquivo_pbit = path.join(caminho_BI, f'{nome_BI}.pbit')
arquivo_zip = path.join(caminho_BI, f'{nome_BI}.zip')
arquivo_json_layout = path.join(caminho_BI, 'Report', 'Layout')
arquivo_json_model = path.join(caminho_BI, 'DataModelSchema')
modelo_word = path.join(caminho_modelo_word, f'{nome_modelo_word}')

In [ ]:
# Verificando se o arquivo_pbix não existe ou se o arquivo_zip já existe
if not path.exists(arquivo_pbit) or path.exists(arquivo_zip):
    # Pule para a próxima instrução
    print("Arquivo .pbit não encontrado ou arquivo .zip já existe. Pulando para a próxima instrução.")
else:
    # Renomeia o arquivo caso a condição seja falsa
    rename(arquivo_pbit, arquivo_zip)

In [ ]:
# Extraindo o arquivo Layout do ZIP
with zipfile.ZipFile(arquivo_zip, 'r') as zip:
    arquivo_json_layout_zip = 'Report/Layout'
    arquivo_json_model_zip = 'DataModelSchema'
    zip.extract(arquivo_json_layout_zip, caminho_BI)
    zip.extract(arquivo_json_model_zip, caminho_BI)

In [ ]:
# Retornando o arquivo zip para pbit
rename(arquivo_zip, arquivo_pbit)

In [ ]:
# Carregando o arquivo Layout em JSON
with open(arquivo_json_layout, 'r', encoding='utf-16-le') as f:
    dados = json.load(f)

In [ ]:
#Extraindo informações dos nomes das páginas do arquivo Layout
display_names = []

for section in dados.get('sections', []):
    display_name = section.get('displayName')
    display_names.append({'Páginas': display_name})

# Convertendo os dados para um DataFrame usando pandas
pages = pd.DataFrame(display_names)

#Exibindo a tabela
display(pages)

In [ ]:
# Extraindo informações dos visuais do arquivo Layout
visual_containers = []

# Extraindo nomes das páginas
for section in dados.get('sections', []):
    display_name = section.get('displayName', 'Sem Nome')

    # Percorre cada visualContainer dentro da seção
    for container in section.get("visualContainers", []):
        # Extrai os valores de x e y do position
        config = json.loads(container.get("config", "{}"))
        for layout in config.get("layouts", []):
            position = layout.get("position", {})
            x = position.get("x")
            y = position.get("y")
            width = position.get("width")
            height= position.get("height")

        # Extrai o visualType e queryRef
        single_visual = config.get("singleVisual", {})
        visual_type = single_visual.get("visualType")

        
        # Coleta todos os queryRefs presentes em qualquer chave de projections
        projections = single_visual.get("projections", {})
        query_refs = []
        for key, items in projections.items():
            for item in items:
                query_ref = item.get("queryRef")
                if query_ref:
                    #Retira a informação da tabela do nome da medida
                    measure_name = query_ref.split(".")[-1]
                    query_refs.append(measure_name)


        # Define query_refs como ""Não há medidas utilizadas no visual" se não houver medidas
        query_refs = query_refs if query_refs else "Não há medidas utilizadas no visual"

        # Adiciona as informações dos visuais à lista, incluindo Página, posição X e Y, tipo de visual e medidas utilizadas
        visual_containers.append({
            "Página": display_name,
            "X": int(x),
            "Y": int(y),
            "Altura": int(height),
            "Largura": int(width),
            "Tipo de visual": visual_type,
            "Medidas utilizadas": query_refs
        })

# Convertendo os dados para um DataFrame usando pandas
visuals = pd.DataFrame(visual_containers)

# Exibindo a tabela
display(visuals)

In [ ]:
#Gera as informações das tabelas do relatório

# Carregando o arquivo ModelShema em JSON
with open(arquivo_json_model, 'r', encoding='utf-16-le') as f:
    dados_model = json.load(f)

#Extraindo nome das tabelas e colunas relacionadas
tables = []

#Extraindo nome das tabelas
for table in dados_model.get('model', {}).get('tables', []):
    table_name = (table.get("name", ""))

    #Extraindo colunas relacionadas
    for column in table.get('columns', []):
        column_name = (column.get("name", ""))
        dataType = (column.get('dataType', ""))
        column_type = (column.get('type', ""))

        # Verificação se o tipo de coluna é calculada ou não
        is_calculated = column_type in ['calculatedTableColumn', 'calculated']

        # Adiciona as informações extraídas à lista, incluindo tabela, nome das colunas, tipo de dados e se é coluna calculada
        tables.append({
        'Tabelas': table_name,
        'Colunas': column_name,
        'Tipo de dados': dataType,
        'Coluna calculada?': 'Sim' if is_calculated else 'Não'
        })
       
# Convertendo os dados para um DataFrame usando pandas
tables = pd.DataFrame(tables)

#Imprimindo para conferência
display(tables)

In [ ]:
# Gerar as medidas utilizadas e seus cálculos

measures = []

#Extraindo nome das tabelas
for table in dados_model.get('model', {}).get('tables', []):

    for measure in table.get('measures',[]):
        measure_name = measure.get('name')
        measure_expression = measure.get('expression')

        # Verifique se a expressão é uma lista e, se sim, una os elementos em uma string
        if isinstance(measure_expression, list):
            measure_expression = '\n'.join(filter(lambda x: x.strip(), measure_expression))

        # Limpeza da expressão para garantir que não tenha quebras de linha indesejadas
        measure_expression = measure_expression.replace('\n', ' ')  # Substitui quebras de linha por espaço

        # Adiciona as informações extraídas à lista, incluindo tabela, nome das colunas, tipo de dados e se é coluna calculada
        measures.append({
            'Medida': measure_name,
            'Expressão': measure_expression
        })
       
# Convertendo os dados para um DataFrame usando pandas
measures = pd.DataFrame(measures)

#Imprimindo para conferência
display(measures)

In [ ]:
# Gerar as fontes das tabelas utilizadas

fonts = []

#Extraindo nome das tabelas
for table in dados_model.get('model', {}).get('tables', []):

    for partition in table.get('partitions', []):
        partition_name = partition.get('name')
        partition_mode = partition.get('mode')

        # Extraindo o tipo de partição e a expressão
        source = partition.get('source', {})
        font_type = source.get('type')
        font_expression = source.get('expression')

        # Adiciona as informações extraídas à lista, incluindo tabela, nome das colunas, tipo de dados e se é coluna calculada
        # Verifique se a expressão é uma lista e, se sim, una os elementos em uma string
        if isinstance(font_expression, list):
            # Remove elementos vazios antes de juntar
            font_expression = '\n'.join(filter(lambda x: x.strip(), font_expression))
        
        fonts.append({
            'Tabela': partition_name,
            'Modo de importação': partition_mode,
            'Tipo de importação': font_type,
            'Fonte': font_expression
        })
       
# Convertendo os dados para um DataFrame usando pandas
fonts = pd.DataFrame(fonts)

#Imprimindo para conferência
display(fonts)

In [ ]:
# Gera os relacionamentos entre as tabelas

relationships = []

#Extraindo os relacionamentos direto do JSON
for relation in dados_model.get('model', {}).get('relationships', []):
    #Puxando as informações
    relationship_fromTable = relation.get('fromTable')
    relationship_toTable = relation.get('toTable')
    relationship_fromColumn = relation.get('fromColumn')
    relationship_toColumn = relation.get('toColumn')

        #Criando as colunas da tabela
    relationships.append({
        'Da tabela': relationship_fromTable,
        'Para tabela': relationship_toTable,
        'Da coluna': relationship_fromColumn,
        'Para coluna': relationship_toColumn
    })
       
# Convertendo os dados para um DataFrame usando pandas
relationships = pd.DataFrame(relationships)

#Imprimindo para conferência
display(relationships)

## Segunda parte do código:

Para a segunda parte do código, utilizo a biblioteca docx para:
1. Abrir o arquivo do modelo word
2. Transformar os dataframes gerados anteriormente em tabelas
3. Incluir cada tabela após o parágrafo correspondente, visando a organização da documentação
4. Salvar o arquivo com o nome do relatório + "_doc" no formato .docx

In [ ]:
# Inserir tabelas no Word

# Nomeando os arquivos do Word e salvando em uma variável
nome_word = nome_BI + '_doc.docx'
salvar_word = path.join(caminho_modelo_word, nome_word)

# Abrir o modelo do Word
document = Document(modelo_word)

# Função para inserir tabela após o parágrafo
def inserir_tabela_apos_paragrafo(paragrafo, dataframe):
    df_columns = dataframe.columns
    # Criar a tabela
    tabela = document.add_table(rows=1, cols=len(df_columns))
    tabela.style = 'Table Grid'

    # Adicionar o cabeçalho da tabela
    for idx, col_name in enumerate(df_columns):
        cell = tabela.rows[0].cells[idx]
        cell.text = col_name
        cell.paragraphs[0].runs[0].font.bold = True  # Deixar o cabeçalho em negrito

    # Adicionar dados à tabela
    for row_data in dataframe.itertuples(index=False):
        row_cells = tabela.add_row().cells
        for idx, value in enumerate(row_data):
            row_cells[idx].text = str(value)

    # Criar um novo parágrafo vazio para a tabela, se necessário
    new_paragraph = document.add_paragraph()
    # Adiciona a tabela após o parágrafo atual
    document._element.body.insert(document._element.body.index(paragrafo._element) + 1, tabela._element)

# Localizar o parágrafo com o título "Data da documentação" e inserir data atual
for para in document.paragraphs:
    if "Data da documentação:" in para.text:
        # Localiza o parágrafo "Data:" e insere a data formatada diretamente nele
        para.add_run(f" {data_formatada}")
        break

# Localizar o parágrafo com o título "Nome do Relatório" e inserir o nome do relatório
for para in document.paragraphs:
    if "Nome do relatório:" in para.text:
        # Insere o nome do arquivo na documentação
        para.add_run(f" {nome_BI}")
        break

# Localizar o parágrafo com o título "Páginas" e inserir tabela
for para in document.paragraphs:
    if para.text == "Páginas":
        inserir_tabela_apos_paragrafo(para, pages)

# Localizar o parágrafo com o título "Tabelas" e inserir tabela
for para in document.paragraphs:
    if para.text == "Tabelas":
        inserir_tabela_apos_paragrafo(para, tables)

# Localizar o parágrafo com o título "Medidas" e inserir tabela
for para in document.paragraphs:
    if para.text == "Medidas":
        inserir_tabela_apos_paragrafo(para, measures)

# Localizar o parágrafo com o título "Visuais" e inserir tabela
for para in document.paragraphs:
    if para.text == "Visuais":
        inserir_tabela_apos_paragrafo(para, visuals)

# Localizar o parágrafo com o título "Fontes" e inserir tabela
for para in document.paragraphs:
    if para.text == "Fontes":
        inserir_tabela_apos_paragrafo(para, fonts)

# Localizar o parágrafo com o título "Relacionamentos" e inserir tabela
for para in document.paragraphs:
    if para.text == "Relacionamentos":
        inserir_tabela_apos_paragrafo(para, relationships)

# Salvar o arquivo
document.save(salvar_word)

print('Documentação gerada!')